In [1]:
import random

In [2]:
from collections import namedtuple

HeapElement = namedtuple('HeapElement', ['priority', 'key'])

def Lofinx(inx):
    """return the lavel of heap depth for index"""
    
    return (inx+1).bit_length()-1
  
    
def firstinL(L):
    """return the first index in the heap layer"""
    
    return 2**L-1


def parentinx(inx):
    """return the index of the parent node in the
        index belongs to root return negative value"""
    
    L = Lofinx(inx)
    return firstinL(L-1)+int((inx-firstinL(L))/2)


def childinxs(inx):
    """return the indexes of the cildren nodes,
        may be greater than the size of the heap"""
    
    L = Lofinx(inx)
    ch1 = firstinL(L+1)+2*(inx-firstinL(L))
    ch2 = ch1+1
    return ch1, ch2


class PriorityQueue:
    
    def __init__(self):
        self.heap = []
        self.indexes = {}
        
        
    def __add_to_heap__(self, priority, key):
        """Add value to the heap, key mast not be in the heap."""
        
        value = HeapElement(priority, key)
        self.heap.append(value)
        self.indexes[value.key] = len(self.heap)-1
        self.__restore_heap_up__(len(self.heap)-1)
        
        
    def __swap__(self,inx1,inx2):
        """Swap two elements in the heap. Heap is not consist after swapping."""
        
        val1 = self.heap[inx1]
        val2 = self.heap[inx2]
        self.heap[inx1] = val2
        self.heap[inx2] = val1
        self.indexes[val2.key]=inx1
        self.indexes[val1.key]=inx2
    
    
    def __restore_heap_down__(self, index):
        """Restore heap recursevly startind from this index,
            this node swaps with the child with higрest priority"""
        
        inx_of_max = index
        ch1,ch2 = childinxs(index)
        
        if ch1<len(self.heap):
            if self.heap[ch1].priority > self.heap[inx_of_max].priority:
                inx_of_max = ch1
                
        if ch2<len(self.heap):
            if self.heap[ch2].priority > self.heap[inx_of_max].priority:
                inx_of_max = ch2

        if inx_of_max != index:
            self.__swap__(inx_of_max,index)
            self.__restore_heap_down__(inx_of_max)
                
        
    def __restore_heap_up__(self,index):
        """Restore heap recursevly startind from this index,
            this node swaps with the parent node if it has higher priority"""
        
        if index > 0:
            pinx = parentinx(index)
            if self.heap[index].priority > self.heap[pinx].priority:
                self.__swap__(index,pinx)
                self.__restore_heap_up__(pinx)
            
            
    def removekey(self, key):
        """Remove this key from heap"""
        if key not in self.indexes:
            return
        
        index = self.indexes[key]
        while index != 0:
            pinx = parentinx(index)
            self.__swap__(pinx,index) 
            index = pinx
        self.pop()
    
    
    def push(self, priority, key):
        """Add element into heap, if this key already in the heap and priority in the heap is lower,
        the new priority set up, if the priority less then existant nothing happen."""
        
        if key in self.indexes:
            heap_inx = self.indexes[key]
            if self.heap[heap_inx].priority <= priority:
                self.removekey(key)
            else:
                return
        self.__add_to_heap__(priority, key)
    
    
    def pop(self):
        """Return the key with the greatest prioryty"""
        if len(self.heap):
            self.__swap__(0,len(self.heap)-1)
            value = self.heap.pop()
            del self.indexes[value.key]
            if len(self.heap):
                self.__restore_heap_down__(0)
                
            return value.key
        else:
            raise IndexError()
            
            
    def size(self):
        return len(self.heap)
        

In [3]:
# test without priority changing
l = [ random.randint(-200,200) for i in range(200)]
l = list(set(l))
random.shuffle(l)

p=PriorityQueue()
for i in l: p.push(i,i*10)
[ sorted(l)[i] for i in range(len(l)-1,-1,-1) ] == [ p.pop()/10 for i in range(len(l))] 

True

In [4]:
l = [ random.randint(-200,200) for i in range(200)]
l = list(set(l))
random.shuffle(l)

p=PriorityQueue()
for i in l: p.push(i,i*10)

list_with_priority = [(i,i*10) for i in l ]
max_prior = max(l)
#make 20 random priority chandges
for i in range(20):
    max_prior += 1
    random_inx = random.randint(0, len(l)-1)
    val = list_with_priority[random_inx]
    list_with_priority[random_inx] = (max_prior,val[1])
    p.push(max_prior,val[1])

l1 = [ p.pop() for i in range(len(l))]

l2 = [ sorted(list_with_priority,key = lambda x:x[0])[i][1] for i in range(len(l)-1,-1,-1) ]

l1 == l2


True

In [5]:
#example the dijkstra path
Edge = namedtuple('Edge', ['fr','to','weight'])

class Node:
    """Node of undirected graph"""
    
    def __init__(self, number):
        self.number = number
        self.edges = []
        
    def add_edge(self, edge):
        self.edges.append((edge.to, edge.weight))
        edge.to.edges.append((edge.fr, edge.weight))
        
        
nodes = [ Node(i) for i in range(100) ]

edges = []
#connected core
for i in range(1,len(nodes)-2):
    for j in range(i+1, len(nodes)-1):
        if random.random() > 0.80:
            edges.append(Edge(nodes[i],nodes[j],random.random()*100))

#sourse connections
source_connected_to = [ random.randint(1,len(nodes)-1) for i in range(5) ]

#sink connections
sink_connected_to = []
while len(sink_connected_to) < 5:
    connection = random.randint(1,len(nodes)-1)
    if connection not in source_connected_to:
        sink_connected_to.append(connection)

for connection in sink_connected_to:
    edges.append(Edge(nodes[connection],nodes[len(nodes)-1],random.random()*100))

for connection in source_connected_to:
    edges.append(Edge(nodes[0],nodes[connection],random.random()*100))    
    
for edge in edges:
    nodes[edge.fr.number].add_edge(edge)

In [6]:
#find the shortest path
fr = 0
destination = len(nodes)-1

road_len_to_distination = None

road_len_to = [ None for i in range(len(nodes)) ]
road_len_to[fr] = 0

nodes_que = PriorityQueue()
nodes_que.push(0,nodes[fr])

while nodes_que.size():
    node = nodes_que.pop()
    
    for edge in node.edges:
        to, weight = edge
        new_road_len = road_len_to[node.number] + weight
        
        if road_len_to[to.number] is None or road_len_to[to.number] > new_road_len:
            road_len_to[to.number] = new_road_len
            nodes_que.push(-new_road_len, to)
    
    if road_len_to_distination != road_len_to[destination]:
        road_len_to_distination = road_len_to[destination]
        print(road_len_to_distination)
    
            
        

81.73340676913189
